In [ ]:
from enum import Enum, IntEnum
from typing import (
    Tuple,
    List,
    TypeVar,
    Iterable,
    Sequence,
    Generic,
    Callable,
    Set,
    Deque,
    Dict,
    Any,
    Optional,
    NamedTuple
)
from typing_extensions import Protocol
import random
import sys
import bisect
from heapq import heappush, heappop
from math import sqrt

# Storing DNA

In [ ]:
Nucleotide: IntEnum = IntEnum('Nucleotide', ('A', 'G', 'T', 'C'))
Codon = Tuple[Nucleotide, Nucleotide, Nucleotide]
Gene = List[Codon]

In [ ]:
gene_str = ''.join([random.choice('AGTC') for _ in range(10000)])

In [ ]:
def str_to_gene(gene_str: str) -> Gene:
    gene: Gene = []
    for i in range(0, len(gene_str), 3):
        if i + 2 >= len(gene_str):
            return gene
        codon: Codon = (Nucleotide[gene_str[i]], Nucleotide[gene_str[i + 1]], Nucleotide[gene_str[i + 2]])
        gene.append(codon)
    return gene

In [ ]:
gene = str_to_gene(gene_str)

In [ ]:
def linear_search(gene: Gene, codon: Codon) -> bool:
    for i in gene:
        if i == codon:
            return True
    return False

In [ ]:
%timeit linear_search(gene, (Nucleotide['A'], Nucleotide['T'], Nucleotide['G']))

# Binary search

In [ ]:
def binary_search(gene: Gene, codon: Codon) -> bool:
    low = 0
    high = len(gene) - 1
    while low <= high:
        mid = (low + high) // 2
        if codon < gene[mid]:
            high = mid - 1
        elif codon > gene[mid]:
            low = mid + 1
        else:
            return True
    return False

In [ ]:
gene = sorted(gene)

In [ ]:
%timeit binary_search(gene, (Nucleotide['A'], Nucleotide['T'], Nucleotide['G']))

In [ ]:
a = [1, 3, 5, 7, 9]
print(a)
print(bisect.bisect(a, 2))
print(a)
bisect.insort(a, 4)
print(a)

# Generic search

In [ ]:
T = TypeVar('T')
C = TypeVar('C', bound='Comparable')

In [ ]:
def linear_contains(iterable: Iterable[T], key: T) -> bool:
    for i in iterable:
        if i == key:
            return True
    return False

class Comparable(Protocol):
    def __eq__(self, other: Any) -> bool:
        return self == other
        
    def __lt__(self: C, other: C) -> bool:
        return not self > other
        
    def __gt__(self: C, other: C) -> bool:
        return (not self < other) and self != other
        
    def __le__(self:C, other: C) -> bool:
        return self < other or self == other
    
    def __ge__(self: C, other: C) -> bool:
        return not self < other
    
def binary_contains(sequence: Sequence[C], key: C) -> bool:
    low: int = 0
    high: int = len(sequence) - 1
    while low <= high:
        mid: int = (low + high) // 2
        if sequence[mid] < key:
            low = mid + 1
        elif sequence[mid] > key:
            high = mid - 1
        else:
            return True
    return False

In [ ]:
print(linear_contains([5, 10, 15, 15, 20], 5))
print(linear_contains([5, 10, 15, 15, 20], 3))
print(binary_contains(['a', 'b', 'c', 'd', 'e'], 'd'))
print(binary_contains(['a', 'b', 'c', 'd', 'e'], 'z'))

# Maze

In [ ]:
class Cell(str, Enum):
    EMPTY = ' '
    BLOCKED = 'X'
    START = 'S'
    GOAL = 'G'
    PATH = '*'
    
class MazeLocation(NamedTuple):
    row: int
    column: int
        
class Maze:
    def __init__(
        self,
        rows: int = 10,
        columns: int = 10,
        sparseness: float = 0.2,
        start: MazeLocation = MazeLocation(0, 0),
        goal: MazeLocation = MazeLocation(9, 9)
    ) -> None:
        self._rows: int = rows
        self._columns: int = columns
        self.start: MazeLocation = start
        self.goal: MazeLocation = goal
        self._grid: List[List[Cell]] = [[Cell.EMPTY] * columns for _ in range(rows)]
        self._random_fill(rows, columns, sparseness)
        self._grid[start.row][start.column] = Cell.START
        self._grid[goal.row][goal.column] = Cell.GOAL
        
    def _random_fill(self, rows, columns, sparseness):
        for r in range(rows):
            for c in range(columns):
                if random.uniform(0, 1.0) < sparseness:
                    self._grid[r][c] = Cell.BLOCKED
                    
    def __str__(self):
        output: str = ''
        for row in self._grid:
            output += ''.join([c for c in row]) + '\n'
        return output
    
    def reached_goal(self, location: MazeLocation) -> bool:
        return location == self.goal
    
    def successors(self, loc: MazeLocation) -> List[MazeLocation]:
        locations = []
        if loc.row + 1 < self._rows and self._grid[loc.row + 1][loc.column] != Cell.BLOCKED:
            locations.append(MazeLocation(loc.row + 1, loc.column))
        if loc.row - 1 >= 0 and self._grid[loc.row - 1][loc.column] != Cell.BLOCKED:
            locations.append(MazeLocation(loc.row - 1, loc.column))
        if loc.column + 1 < self._columns and self._grid[loc.row][loc.column + 1] != Cell.BLOCKED:
            locations.append(MazeLocation(loc.row, loc.column + 1))
        if loc.column - 1 >= 0 and self._grid[loc.row][loc.column - 1] != Cell.BLOCKED:
            locations.append(MazeLocation(loc.row, loc.column - 1))
        return locations
    
    def mark(self, path: List[MazeLocation]) -> None:
        for loc in path:
            self._grid[loc.row][loc.column] = Cell.PATH
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL
        
    def clear(self, path: List[MazeLocation]):
        for loc in path:
            self._grid[loc.row][loc.column] = Cell.EMPTY
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL

In [ ]:
maze = Maze()

In [ ]:
print(maze)

In [ ]:
class Stack(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []
    
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item: T) -> None:
        self._container.append(item)
        
    def pop(self) -> T:
        return self._container.pop()
    
    def __repr__(self) -> str:
        return repr(self._container)

In [ ]:
class Node(Generic[T]):
    def __init__(
        self,
        state: T,
        parent, # Optional[Node]
        cost: float = 0.0,
        heuristic: float = 0.0
    ) -> None:
        self.state = state
        self.parent = parent
        self.cost = cost
        self.heuristic = heuristic
        
    def __lt__(self, other) -> bool:
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

In [ ]:
def dfs(
    initial: T,
    goal_test: Callable[[T], bool],
    successors: Callable[[T], List[T]]
) -> Optional[Node[T]]:
    
    # where we have yet to go
    frontier: Stack[Node[T]] = Stack()
    frontier.push(Node(initial, None))
    
    # where we have been
    explored: Set[T] = {initial}

        
    # keep going while there is more to explore
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
            
        # if we find the goal we are done
        if goal_test(current_state):
            return current_node
        
        # check were we can go and have not explored
        for child in successors(current_state):
            if child in explored:
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    
    return

In [ ]:
class Queue(Generic[T]):
    def __init__(self) -> None:
        self._container: Deque[T] = Deque()
            
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item: T) -> None:
        self._container.append(item)
        
    def pop(self) -> T:
        return self._container.popleft()
    
    def __repr__(self) -> str:
        return repr(self._container)

In [ ]:
def bfs(
    initial: T,
    goal_test: Callable[[T], bool],
    successors: Callable[[T], List[T]]
) -> Optional[Node[T]]:
    
    # where we have yet to go
    frontier: Queue[Node[T]] = Queue()
    frontier.push(Node(initial, None))
    
    # where we have been
    explored: Set[T] = {initial}
        
    count = 0
        
    # keep going while there is more to explore
    while not frontier.empty:
        count += 1
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
            
        # if we find the goal we are done
        if goal_test(current_state):
            return current_node, count
        
        # check were we can go and have not explored
        for child in successors(current_state):
            if child in explored:
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    
    return None, None

In [ ]:
def len_path(node):
    counter = 1
    path = [node.state]
    while node.parent is not None:
        counter += 1
        node = node.parent
    return counter

In [ ]:
def node_to_path(node: Node[T]) -> List[T]:

    path: List[T] = [node.state]
    
    # work backwards from end to head of linked list
    while node.parent is not None:
        node = node.parent
        path.append(node.state)
        
    path.reverse()
    return path

In [ ]:
m = Maze(rows=25, columns=100, goal=MazeLocation(24, 99), sparseness=0.15)
solution, count = bfs(m.start, m.reached_goal, m.successors)
if solution is None:
    print('No solution found using depth-first search')
else:
    path = node_to_path(solution)
    m.mark(path)
    print(m)
    m.clear(path)

In [ ]:
class PriorityQueue(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []
        
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item: T) -> None:
        heappush(self._container, item)
        
    def pop(self) -> T:
        return heappop(self._container)
        
    def __repr__(self) -> str:
        return repr(self._container)

In [ ]:
def euclidean_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(loc: MazeLocation) -> float:
        xdist: int = loc.column - goal.column
        ydist: int = loc.row - goal.row
        return sqrt((xdist ** 2) + (ydist ** 2))
    return distance

def manhattan_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(loc: MazeLocation) -> float:
        xdist: int = abs(loc.column - goal.column)
        ydist: int = abs(loc.row - goal.row)
        return xdist + ydist
    return distance

In [ ]:
def astar(
    initial: T,
    goal_test: Callable[[T], bool],
    successors: Callable[[T], List[T]],
    heuristic: Callable[[T], float]
):
    frontier: PriorityQueue[Node[T]] = PriorityQueue()
    frontier.push(Node(initial, None, 0.0, heuristic(initial)))
    explored: Dict[T, float] = {initial: 0.0}
        
    count = 0
    
    while not frontier.empty:
        count += 1
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        
        if goal_test(current_state):
            return current_node, count
        
        for child in successors(current_state):
            new_cost: float = current_node.cost + 1
            if child not in explored or explored[child] > new_cost:
                explored[child] = new_cost
                frontier.push(Node(child, current_node, new_cost, heuristic(child)))
                
    return None, None

In [ ]:
steps_dfs = []
steps_bfs = []
counts_bfs = []
steps_astar = []
counts_astar = []

for i in range(100):
    m = Maze(rows=25, columns=100, goal=MazeLocation(24, 99), sparseness=0.15)

    solution_dfs = dfs(m.start, m.reached_goal, m.successors)
    if solution_dfs is not None:
        steps_dfs.append(len_path(solution_dfs))
        
    solution_bfs, count_bfs = bfs(m.start, m.reached_goal, m.successors)
    if solution_bfs is not None:
        steps_bfs.append(len_path(solution_bfs))
        counts_bfs.append(count_bfs)
    
    distance: Callable[[MazeLocation], float] = manhattan_distance(m.goal)
    solution_astar, count_astar = astar(m.start, m.reached_goal, m.successors, distance)
    if solution_astar is not None:
        steps_astar.append(len_path(solution_astar))
        counts_astar.append(count_astar)
        
mean_dfs = sum(steps_dfs) / len(steps_dfs)
mean_bfs = sum(steps_bfs) / len(steps_bfs)
mean_astar = sum(steps_astar) / len(steps_astar)

print('Mean steps DFS: ', mean_dfs)
print()
print('Mean steps BFS: ', mean_bfs)
print('Mean count BFS: ', sum(counts_bfs) / len(counts_bfs))
print()
print('Mean steps A*: ', mean_astar)
print('Mean count A*: ', sum(counts_astar) / len(counts_astar))

In [ ]:
m = Maze(rows=25, columns=100, goal=MazeLocation(24, 99), sparseness=0.15)
distance: Callable[[MazeLocation], float] = manhattan_distance(m.goal)
solution3, count = astar(m.start, m.reached_goal, m.successors, distance)
if solution3 is None:
    print("No solution found using A*.")
else:
    path3: List[MazeLocation] = node_to_path(solution3)
    m.mark(path3)
    print(m)
    m.clear(path)

# Missionaries and cannibals

In [ ]:
MAX_NUM = 3

class MCState:
    def __init__(
        self,
        missionaries: int,
        cannibals: int,
        boat: bool
    ) -> None:
        self.wm: int = missionaries  # missionaries on the west bank
        self.wc: int = cannibals  # cannibals on the west bank
        self.em: int = MAX_NUM - self.wm  # missionaries on the east bank
        self.ec: int = MAX_NUM - self.wc  # cannibals on the east bank
        self.boat: bool = boat
            
    def __str__(self):
        return (
            f'On the west bank there are {self.wm} missionaries and {self.wc} cannibals.\n'
            f'On the east bank there are {self.em} missionaries and {self.ec} cannibals.\n'
            f'The boat is on the {"west" if self.boat else "east"} bank.\n'
        )
              
    def goal_test(self):
        return self.is_legal and self.em == MAX_NUM and self.ec == MAX_NUM
              
    @property
    def is_legal(self):
        if self.wm > 0 and self.wm < self.wc:
            return False
        if self.em > 0 and self.em < self.ec:
            return False
        return True
              
    def successors(self) -> List[MCState]:
        s: List[MCState] = []
        # boat on the west bank
        if self.boat:
            if self.wm > 1:
                s.append(MCState(self.wm - 2, self.wc, not self.boat))
            if self.wm > 0:
                s.append(MCState(self.wm - 1, self.wc, not self.boat))
            if self.wc > 1:
                s.append(MCState(self.wm, self.wc - 2, not self.boat))
            if self.wc > 0:
                s.append(MCState(self.wm, self.wc - 1, not self.boat))
            if (self.wc > 0) and (self.wm > 0):
                s.append(MCState(self.wm - 1, self.wc - 1, not self.boat))
        else:
            if self.em > 1:
                s.append(MCState(self.wm + 2, self.wc, not self.boat))
            if self.em > 0:
                s.append(MCState(self.wm + 1, self.wc, not self.boat))
            if self.ec > 1:
                s.append(MCState(self.wm, self.wc + 2, not self.boat))
            if self.ec > 0:
                s.append(MCState(self.wm, self.wc + 1, not self.boat))
            if (self.ec > 0) and (self.em > 0):
                s.append(MCState(self.wm + 1, self.wc + 1, not self.boat))
        return [x for x in s if x.is_legal]

In [ ]:
def display_solution(path: List[MCState]):
    if len(path) == 0:
        return
    old: MCState = path[0]
    print(old)
    for current in path[1:]:
        if current.boat:
            print(f'{old.em - current.em} missionaries and {old.ec - current.ec} cannibals moved to west bank.')
        else:
            print(f'{old.wm - current.wm} missionaries and {old.wc - current.wc} cannibals move to east bank.')
        print(current)
        old = current

In [ ]:
start: MCState = MCState(MAX_NUM, MAX_NUM, True)
solution, count = bfs(start, MCState.goal_test, MCState.successors)
if solution is None:
    print('No solution found.')
else:
    path: List[MCState] = node_to_path(solution)
    display_solution(path)